In [45]:
import re
import os
import glob
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
 

In [46]:
def sort_dirs(path):
    path = os.path.basename(path)
    if path.startswith("."):
        return 0
    found = re.search(r'\((\d+)\)', path)
    if not found:
        return 0
    return int(found.group(1))

def sort_prod(file):
    if file.startswith("."):
        return 0
    try:
        found = re.search(r'2022-\d{2}-\d{2}', file)
        index = found.end()
    except Exception as e:
        print(f"ERR on {file}")
    return int(file[index - 2:file.index(".csv")])
    
def split_time_day(df):
    df[['time','day']] = (
        df['time_day']
           .str.split('/', expand=True)
           .apply(lambda col: col.str.strip())
    )
    df['day'] = df['day'].astype(int)
    df.drop('time_day', axis=1, inplace=True)
    return df

def parse_conditions(dir):
    path = os.path.join(os.getcwd(), dir)
    # Get diagram 1 data into a dataframe
    full_path = os.path.join(path, "Weather_Diagram_1*")
    diagram_1_data = []
    for file in sorted(glob.glob(full_path), key=sort_dirs):
        if file.startswith("."):
            continue
        data = pd.read_csv(file, sep=";",skiprows=1,header=None,
                           names=["time_day","ambient","module_temp","wind"])
        diagram_1_data.append(data)
    df1 = pd.concat(diagram_1_data, ignore_index=True)
    df1 = split_time_day(df1)
    
    # Get diagram 2 data into a dataframe
    full_path = os.path.join(path, "Weather_Diagram_2*")
    diagram_2_data = []
    for file in sorted(glob.glob(full_path), key=sort_dirs):
        if file.startswith("."):
            continue
        data = pd.read_csv(file, sep=";",skiprows=1,header=None,
                           names=["time_day","insolation"])
        diagram_2_data.append(data)
    df2 = pd.concat(diagram_2_data, ignore_index=True)
    df2 = split_time_day(df2)
    
    # Merge into one dataframe
    combined_df = pd.merge(df1,df2,on=['day','time'])
    return combined_df
    
def parse_production(dir):
    path = os.path.join(os.getcwd(), dir)
    df = []
    for file in sorted(os.listdir(path),key=sort_prod):
        if file.startswith("."):
            continue
        full_path = f"{path}/{file}"
        data = pd.read_csv(full_path, sep=";",skiprows=1,header=None, 
                           names=["time","power"])
        day = int(file.split("-")[-1].split(".")[0])
        data['day'] = day
        df.append(data)
    df = pd.concat(df, ignore_index=True)
    df['time'] = df['time'].str.strip()
    df['power'] = pd.to_numeric(df['power'], errors='coerce')
    return df
    
    
        
# Load data
data_dir = "PVSystem/"

all_inputs = []
all_outputs = []
for dir in sorted(os.listdir(data_dir)):
    full_path = os.path.join(data_dir, dir)
    if not os.path.isdir(full_path) or dir.startswith("."):
        continue
    cond_df = parse_conditions(full_path + '/Conditions')
    prod_df = parse_production(full_path + '/Production')
    all_inputs.append(cond_df.assign(month=dir))
    all_outputs.append(prod_df.assign(month=dir))

all_inputs = pd.concat(all_inputs, ignore_index=True)
all_outputs = pd.concat(all_outputs, ignore_index=True)
all_data = pd.merge(
    all_inputs,
    all_outputs,
    on=['month','day','time'],
    how='left'
)


In [47]:
# Create and train model
all_data['hour'] = all_data['time'].str.extract(r'(\d+):').astype(int)
all_data['minute'] = all_data['time'].str.extract(r':(\d+)').astype(int)
features = ['ambient', 'module_temp', 'wind', 'insolation', 'hour', 'minute', 'day']
target = ['power']

data = all_data[features + target].dropna().reset_index(drop=True)

scaler_X = MinMaxScaler()
scaler_Y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(data[features])
Y_scaled = scaler_Y.fit_transform(data[target])

SEQ_LEN = 12
def create_sequences(X, Y, seq_len):
    Xs, Ys = [], []
    for i in range(len(X) - seq_len):
        Xs.append(X[i:i+seq_len])
        Ys.append(Y[i+seq_len])
    return np.array(Xs), np.array(Ys)

X_seq, y_seq = create_sequences(X_scaled, Y_scaled, SEQ_LEN)
X_train, X_val, Y_train, Y_val = train_test_split(X_seq, y_seq, test_size=0.2,
                                                  random_state=42,shuffle=False)

tf.random.set_seed(42)
model = Sequential([
    Input(shape=(SEQ_LEN, X_seq.shape[2])),
    LSTM(64, recurrent_activation='sigmoid',activation='tanh'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mse'])
log_dir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
earlystopping_callback = EarlyStopping(patience=20, restore_best_weights=True)


history = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    epochs=500,
    batch_size=64,
    callbacks=[earlystopping_callback, tensorboard_callback]
)

model.evaluate(X_val, Y_val)

Epoch 1/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0710 - mse: 0.0710 - val_loss: 0.0308 - val_mse: 0.0308
Epoch 2/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0327 - mse: 0.0327 - val_loss: 0.0218 - val_mse: 0.0218
Epoch 3/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0251 - mse: 0.0251 - val_loss: 0.0179 - val_mse: 0.0179
Epoch 4/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0233 - mse: 0.0233 - val_loss: 0.0148 - val_mse: 0.0148
Epoch 5/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0225 - mse: 0.0225 - val_loss: 0.0144 - val_mse: 0.0144
Epoch 6/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0209 - mse: 0.0209 - val_loss: 0.0142 - val_mse: 0.0142
Epoch 7/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0199 - mse: 0.0199 - val_loss: 0.0136 - val_mse: 0.0136
Epoch 8/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0203 - mse: 0.0203 - val_loss: 0.0134 - val_mse: 0.0134
Epoch 9/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.019

[0.008980533108115196, 0.008980533108115196]

In [49]:
avg = []
for xi,xp, y in zip(X_val, model(X_val), Y_val):
    err = (1 - (abs((xp - y)))) * 100
    avg.append(err)
print(f"Avg acc: {np.mean(avg)}")

Avg acc: 93.70675659179688
